In [1]:
%matplotlib inline
import os
from osgeo import gdal
import rasterio
import geopandas as gpd
import rasterio.mask
from rasterio.windows import Window
import sys
from shapely.geometry import mapping
sys.path.append(r'C:\Users\ruben\Desktop\GEM\geoloc2\Detecciondeterrenos')
from codigos import Generar_txt
###path de yolo dentro de computadora
os.chdir(r'C:\Users\ruben\yolov7')
from detect_Alberto_v4 import *
from scipy.ndimage import rotate as rotate_image
from shapely import geometry
import time
import datetime
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import math
from shapely.geometry import Polygon
import cv2
from clasificacion_chinchetas import *
import tqdm
import numpy as np
from osgeo import gdal
import rasterio
import geopandas as gpd
import rasterio.mask
import sys
from shapely.geometry import mapping
from shapely import geometry
import cv2
import numpy as np

In [ ]:
def get_lados_xy(poligono, n_digits:int=4):
    
    '''
(Function)
    Esta funcion calcula la distancia del poligono asumiendo que es un paralelogramo, es decir toma el valor minimo en el eje x y el maximo
    para obtener la distancia en x, y lo mismo en el eje y. No importa que tipo de poligono le pases, siempre returna 2 valores, el valor del
    lado x, y el del lado y.
    Muy importante, asegurarse que su crs sea 3857, de lo contrario, los valores retornados seran otra interpretacion.
(Parameters):
    - poligono: corresponde a un registro de la GeoSerie geometry
    - n_digits: cantidad de digitos a la que se redondeara la distancia
    '''
#     if isinstance(poligono,Polygon):
    #valores = poligono.bounds
    from shapely.geometry import Point
    vertices = list(poligono.exterior.coords)

    # Calcular la distancia entre los vértices adyacentes
    lado_1 = round(Point(vertices[0]).distance(Point(vertices[1])),n_digits)
    lado_2 = round(Point(vertices[1]).distance(Point(vertices[2])),n_digits)
    return lado_1, lado_2

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jun  1 16:29:52 2023

@author: Alberto
"""
import geopandas as gpd
import rasterio
from affine import Affine
# Open the raster dataset
raster_path = r"C:\Users\ruben\Desktop\GEM\Geo\Rasters\Temoaya.tif"
raster_dataset = rasterio.open(raster_path)

# Open the shapefile and get the geometry
deteccion=r"C:\Users\ruben\Desktop\GEM\Geo\Bases\Delegaciones\TEMOAYA\Zona_Temoaya.shp"
deteccion_shp=gpd.read_file(deteccion).to_crs("3857")
geometry=[]
zona=[]
long1=[]
box_l=[]
for i in tqdm.tqdm(range(len(deteccion_shp))):
    shape=mapping(deteccion_shp["geometry"][i])
    coord=[(x[0],x[1]) for x in shape.get("coordinates")[0]]
    left,bottom=np.min(coord,axis=0)
    rigth,top=np.max(coord,axis=0)
    window=raster_dataset.window(left,bottom,rigth,top)
    band = raster_dataset.read(2,window=window)
    transform=raster_dataset.meta.get("transform")[0]
    width=band.shape[1]
    width_min=left
    width_max=rigth
    heigth=band.shape[0]
    heigth_min=bottom
    height_max=top
    blac=np.zeros_like(band)
    box=np.array([(int(np.round(map_d(x[0],width_min,width_max,0,width),0)),int(np.round(map_d(x[1],heigth_min,height_max,heigth,0),0))) for x in coord])
    po=cv2.fillPoly(blac,[box],(1));
    masked_image=blac*np.array(band,dtype="float32")
    out_transform=Affine((rigth-left)/width,0,left,0,-(top-bottom)/heigth,top)
    img = np.array(masked_image.copy(),dtype="float32")**2
    f=35000
    img[img<f]=0
    img[img>=f]=255
    contours,hister=cv2.findContours(np.array(img,dtype="uint8"),2,cv2.CHAIN_APPROX_SIMPLE)
    contours1=[(len(x),x) for x in contours if len(x)>=3]
    long=np.array(contours1)[:,0]
    cnts=np.array(contours1)[:,1]
    for j,c in enumerate(cnts):
        area=cv2.contourArea(c)
        if area>50:
            rect = cv2.minAreaRect(c)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            box_l.append(Polygon([(map_d(x[0],0,width,width_min,width_max),map_d(x[1],heigth,0,heigth_min,height_max)) for x in box]))
            # cv.drawContours(img,[box],0,(0,0,255),2)
            long1.append(long[j])
            zona.append(deteccion_shp.loc[i,"zona"])
            geometry.append(Polygon([(map_d(x[0][0],0,width,width_min,width_max),map_d(x[0][1],heigth,0,heigth_min,height_max)) for x in c]))


100%|██████████| 12/12 [03:02<00:00, 15.17s/it]


In [3]:
a=gpd.GeoDataFrame({"puntos":long1,"zona":zona},geometry=geometry,crs="3857")
a["puntos"]=a["puntos"].astype("int")
a["area"]=a.area
a["rectangulo"]=gpd.GeoSeries(box_l,crs=3857).area
a.to_file(r"C:\Users\ruben\Desktop\GEM\Geo\Salidas\Industrias\Temoaya_industrias.shp")

b=gpd.GeoDataFrame({"puntos":long1,"zona":zona},geometry=box_l,crs="3857")
b["puntos"]=a["puntos"].astype("int")
b["area"]=a.area
b["rectangulo"]=b.area
b['Medida_lx'] = b['geometry'].map(lambda x: get_lados_xy(x)[0]) 
b['Medida_ly'] = b['geometry'].map(lambda x: get_lados_xy(x)[1])
b['Max_xy'] = b.apply(lambda row: max(row['Medida_ly'], row['Medida_lx']), axis=1)
b['Min_xy'] = b.apply(lambda row: min(row['Medida_ly'], row['Medida_lx']), axis=1)
b['f_lado'] = b['Max_xy']/b['Min_xy']
b['f_area'] = b['area']/b['rectangulo']
b.to_file(r"C:\Users\ruben\Desktop\GEM\Geo\Salidas\Industrias/Temoaya_industrias_c.shp")